In [1]:
# 연관규칙 실습용 필요한 패키지 설치
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np

- 추천 진행할 때 동시구매 빈도, 조건부 확률 개념으로 수치를 계산해서 추천할 때 제공하는 경우가 있다.
- cos 유사도 같은 개념이 아닌, 신뢰지도, 지지도, 향상도 개념으로 계산하여서 추천 수치를 뽑는 방식
- 추천시스템이라고 하는 대표적인 다양한 방법들 중에
- Asscociation_rule 은 그 중 하나인 것 
- 협업 필터링이나, 콘텐츠 기반 필터링, 딥러닝 방법 등도 있고 다양하다. 메타러닝도 있고 등등

## 대표적인 연관규칙 
- Apriori - 이 알고리즘을 사용할 예정 
- FP-Growth
- DHP 

- 연관규칙은 -> A제품 구매한 사람이 있으면 B제품도 구매할 확률이 높다 라는 결과 이끌어내는 것
- 맥주-> 기저귀 
- 귀저귀 -> 맥주 

- 상품들의 연관을 가지고 계산하여서 어떤 상품을 추천할지를 나타내는 것

- Apriori - 아이템들의 조합에 대한 경우의 수 가지고 최소소하여서 계산하는 형태

- 1 : A, B
- 2 : A, B, C
- 3 : A, B, E
- 4 : A, B, C, D, E 등등

- 시퀀스의 개념까지 생각하면 비행기 티켓을 구매한 사람 -> 여행을 가는 것, 렌트카나, 숙소 이런 부분을 추천에 대해서 진행하는 게 아무래도 더 구매할 확률이 높다.
- 콜드스타트문제 : 처음 온 고객 구매에 대한 데이터가 없다? -> 어떤 걸 추천해야 하는지? 



- Apriori 
- 지지도(Support) - 고객이 A와 B를 동시에 거래한 것에 발생할 확률
- 신뢰도(Confidence) - A구매한 주문 B가 포함될 확률
- 향상도(Lift) - A, B가 독립적으로 판매되는 것 대비 함께 판매되는 비율 

In [5]:
df=pd.read_csv('market_basket.csv')

In [8]:
df_sp=df[:20000]


- cust_cd : [A,B,C,D,E] 구매에 대한 데이터셋을 전처리 해야 한다.

In [11]:
# 학습하기 위한 데이터셋 만들기
item_set = df_sp.drop_duplicates(['cust_cd','std_dt','prdct_nm']).groupby(['cust_cd','std_dt'])['prdct_nm'].apply(list)


In [13]:
# 고객이 주문한 상품들을 하나의 데이터프레임으로 만드는 작업
item_set_df = pd.DataFrame(item_set).reset_index().drop(['cust_cd','std_dt'], axis='columns')

In [16]:
it_set=item_set_df.squeeze()

In [19]:
for i in it_set:
    print(i)

['tropical fruit', 'cleaner', 'bottled water', 'chocolate']
['berries', 'coffee', 'herbs', 'beef', 'fruit/vegetable juice', 'cream', 'tropical fruit', 'whole milk', 'detergent', 'chicken']
['specialty bar', 'other vegetables', 'chocolate', 'chicken']
['tropical fruit', 'butter milk', 'sparkling wine', 'chicken', 'bottled water', 'cream']
['packaged fruit/vegetables', 'sausage', 'fruit/vegetable juice', 'tropical fruit', 'pasta']
['pastry', 'fruit/vegetable juice', 'brown bread', 'cream', 'canned beer']
['berries', 'hamburger meat', 'sparkling wine', 'beef', 'pastry', 'dog food', 'cream', 'onions']
['cream', 'pork', 'beef', 'specialty bar', 'pip fruit', 'packaged fruit/vegetables', 'bottled water', 'pastry']
['specialty bar', 'cream', 'pork', 'turkey', 'beef', 'bottled water', 'pastry']
['specialty bar', 'brown bread', 'pork', 'chocolate']
['specialty bar', 'cream', 'beef', 'pip fruit', 'bottled water', 'skin care']
['chicken', 'tropical fruit', 'pot plants', 'root vegetables']
['pastry

- apriori 모델을 학습시키위 한 장바구니 더미변수를 만들어야 한다.

- 전체 주문의 Unique 값으로 정리해서 해당 제품들의 유니크값을 컬럼을 만들어야 한다.
- chicken, specialty bar, cream,
- 0    0.         1       0      0
- 1    0.         0.      0.     0

In [20]:
encoder=TransactionEncoder()

In [23]:
encoder_T =encoder.fit(it_set).transform(it_set)

In [29]:
it_matrix= pd.DataFrame(encoder_T, columns = encoder.columns_) *1

In [30]:
it_matrix

,beef,berries,beverages,bottled beer,bottled water,brown bread,butter,butter milk,canned beer,chicken,...,sparkling wine,specialty bar,specialty chocolate,sugar,syrup,tropical fruit,turkey,white wine,whole milk,yogurt
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,1,...,1,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2911,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2912,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2913,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


- apriori 모델 적용

In [43]:
it_apr = apriori(it_matrix, min_support = 0.05, use_colnames=True)

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [44]:
it_apr

,support,itemsets
0,0.167410,(beef)
1,0.103602,(berries)
2,0.088851,(bottled water)
3,0.054889,(brown bread)
4,0.086106,(butter milk)
...,...,...
60,0.057290,"(sparkling wine, tropical fruit)"
61,0.061063,"(specialty bar, tropical fruit)"
62,0.051458,"(white wine, tropical fruit)"
63,0.104288,"(whole milk, tropical fruit)"


In [47]:
association_rules(it_apr, metric= 'lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(cream),0.167410,0.308062,0.066209,0.395492,1.283807,0.014637,1.144630,0.265517
1,(cream),(beef),0.308062,0.167410,0.066209,0.214922,1.283807,0.014637,1.060519,0.319489
2,(beef),(tropical fruit),0.167410,0.544082,0.093310,0.557377,1.024435,0.002226,1.030036,0.028648
3,(tropical fruit),(beef),0.544082,0.167410,0.093310,0.171501,1.024435,0.002226,1.004937,0.052317
4,(tropical fruit),(berries),0.544082,0.103602,0.053516,0.098361,0.949408,-0.002852,0.994187,-0.104649
5,(berries),(tropical fruit),0.103602,0.544082,0.053516,0.516556,0.949408,-0.002852,0.943063,-0.056111
6,(bottled water),(tropical fruit),0.088851,0.544082,0.052144,0.586873,1.078647,0.003802,1.103576,0.080022
7,(tropical fruit),(bottled water),0.544082,0.088851,0.052144,0.095839,1.078647,0.003802,1.007729,0.159924
8,(cream),(chicken),0.308062,0.271355,0.076844,0.249443,0.919250,-0.006750,0.970806,-0.112651
9,(chicken),(cream),0.271355,0.308062,0.076844,0.283186,0.919250,-0.006750,0.965297,-0.107586


### Auto-ML
- 머신러닝 자동 학습하는 모델링 패키지
- 알아서 학습하고, 최적의 하이퍼파라미터 파이프라인을 공유한다.

- 분류관련 tpot 모델 진행
- 회귀모델도 있다.

In [49]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [50]:
iris = load_iris()

In [52]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.7, test_size=0.3)

In [53]:
X_train

array([[6.8, 2.8, 4.8, 1.4],
       [5.1, 3.3, 1.7, 0.5],
       [6.3, 3.3, 6. , 2.5],
       [6.9, 3.2, 5.7, 2.3],
       [4.4, 2.9, 1.4, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [6.7, 3.1, 5.6, 2.4],
       [6.8, 3. , 5.5, 2.1],
       [6.5, 2.8, 4.6, 1.5],
       [5.5, 2.3, 4. , 1.3],
       [5.1, 3.5, 1.4, 0.2],
       [5.9, 3.2, 4.8, 1.8],
       [5.7, 2.8, 4.5, 1.3],
       [6.6, 3. , 4.4, 1.4],
       [6.8, 3.2, 5.9, 2.3],
       [6.7, 3.3, 5.7, 2.5],
       [6. , 2.9, 4.5, 1.5],
       [5.5, 2.4, 3.8, 1.1],
       [7.3, 2.9, 6.3, 1.8],
       [4.8, 3. , 1.4, 0.1],
       [4.9, 3. , 1.4, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [5. , 3.5, 1.3, 0.3],
       [4.9, 2.4, 3.3, 1. ],
       [6.6, 2.9, 4.6, 1.3],
       [5.8, 2.7, 5.1, 1.9],
       [5.7, 4.4, 1.5, 0.4],
       [5. , 2.3, 3.3, 1. ],
       [5.1, 3.8, 1.5, 0.3],
       [6.3, 2.9, 5.6, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [7.9, 3.8, 6.4, 2. ],
       [6.1, 2.6, 5.6, 1.4],
       [5.4, 3

In [55]:
# 0,1,2 의 다중분류값
# 모델 진행할 때 분류 모델을 진행해야 하고, 다중값이니 -> DT, XGB, LGB, RF 기본적인 분류모델들 생각할 것 -> 이것들을 직접 패키지가 다 돌아보면 가장 좋은 성능이 무엇인지를 우리에게 알려주는 것! 
iris.target
## 해당 모델 사용하고 빠르게 AutoML로 성능보고, 어떤 하이퍼파라미터나 모델이 좋은 성능을 보이는지 확인할 수 있다.

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [60]:
tpot =TPOTClassifier(generations = 5, population_size=20, random_state=111)

In [61]:
tpot.fit(X_train,y_train)

Version 0.12.1 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


TPOTClassifier(generations=5, population_size=20, random_state=111)

In [62]:
print(tpot.score(X_test, y_test)) # test 스코어를 볼 수 있다.

0.9555555555555556


In [64]:
tpot.evaluated_individuals_# 모델과, 하이퍼파라미터 모두 보고 싶은 경우

{'XGBClassifier(input_matrix, XGBClassifier__learning_rate=1.0, XGBClassifier__max_depth=5, XGBClassifier__min_child_weight=20, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.5, XGBClassifier__verbosity=0)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': 0.3333333333333333},
 'GaussianNB(input_matrix)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': 0.9714285714285713},
 'KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=72, KNeighborsClassifier__p=2, KNeighborsClassifier__weights=distance)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': 0.9714285714285713},
 'GaussianNB(SelectFwe(input_matrix, SelectFwe__alpha=0.037))': {'generation': 0,
  'mutation_count':

In [66]:
tpot.fitted_pipeline_ # 가장 좋은 성능의 모델은 어떤 것인지

Pipeline(steps=[('gaussiannb', GaussianNB())])

In [67]:
from tpot import TPOTRegressor # 회귀모델의 경우는 패키지를 사용하면 된다.
# 모델 사용은 동일한 방식 TPOTClassifier